<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/PULP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [14]:
file_path = 'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/FantasyPros_2024_Overall_ADP_Rankings.csv'
df = pd.read_csv(file_path, on_bad_lines='skip')

def standardize_column_names(df):
    def standardize(col):
        col_lower = col.lower()
        if "player" in col_lower:
            return "Player"
        elif "avg" in col_lower:
            return "AVG"
        else:
            return col
    df.columns = [standardize(col) for col in df.columns]
    return df

df = standardize_column_names(df)

k = 0.02
df['Value'] = 1000 * np.exp(-k * (df['AVG'] - 1))
df['POS'] = df['POS'].str.replace('\d+', '', regex=True)
df

,Rank,Player,Team,Bye,POS,ESPN,Sleeper,NFL,RTSports,FFC,AVG,Value
0,1.0,Christian McCaffrey,SF,9,RB,1.0,1.0,1.0,1.0,NaN,1.0,1000.000000
1,2.0,CeeDee Lamb,DAL,7,WR,3.0,3.0,2.0,2.0,NaN,2.6,968.506582
2,3.0,Tyreek Hill,MIA,6,WR,4.0,2.0,3.0,3.0,NaN,3.2,956.953957
3,4.0,Bijan Robinson,ATL,12,RB,2.0,6.0,8.0,4.0,NaN,5.0,923.116346
4,5.0,Breece Hall,NYJ,12,RB,5.0,8.0,7.0,5.0,NaN,5.4,915.760877
...,...,...,...,...,...,...,...,...,...,...,...,...
937,945.0,Mitchell Tinsley,CIN,12,WR,NaN,NaN,895.0,NaN,NaN,895.0,0.000017
938,947.0,Anthony Gould,IND,14,WR,NaN,NaN,899.0,NaN,NaN,899.0,0.000016
939,948.0,Patrick Murtagh,NaN,NaN,TE,NaN,NaN,900.0,NaN,NaN,900.0,0.000016
940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df = df[['Player', 'Bye', 'POS', 'AVG', 'Value']].dropna()

# Only keep relevant positions
df = df[df['POS'].str.contains('QB|RB|WR|TE|K|DST', na=False)]

# NaN-Werte in AVG durch den maximalen Wert +1 ersetzen (als "undraftbare" Spieler markieren)
df['AVG'] = df['AVG'].fillna(df['AVG'].max() + 1)

print(f'Length of df: {len(df)}')
print(f'value count of POS: {df["POS"].value_counts()}')
print(f'df head: {df.head(20)}')

Length of df: 665
value count of POS: POS
WR     244
RB     142
TE     122
QB      89
K       36
DST     32
Name: count, dtype: int64
df head:                  Player Bye POS   AVG        Value
0   Christian McCaffrey  9   RB   1.0  1000.000000
1           CeeDee Lamb   7  WR   2.6   968.506582
2           Tyreek Hill   6  WR   3.2   956.953957
3        Bijan Robinson  12  RB   5.0   923.116346
4           Breece Hall  12  RB   5.4   915.760877
5     Amon-Ra St. Brown   5  WR   6.2   901.225297
6         Ja'Marr Chase  12  WR   6.6   894.044258
7      Justin Jefferson   6  WR   7.0   886.920437
8        Saquon Barkley   5  RB   9.2   848.742022
9            A.J. Brown   5  WR  10.2   831.935804
10      Jonathan Taylor  14  RB  10.4   828.614707
11       Garrett Wilson  12  WR  12.4   796.124260
12         Jahmyr Gibbs   5  RB  12.8   789.780674
13           Puka Nacua   6  WR  14.2   767.973540
14       Kyren Williams   6  RB  16.0   740.818221
15  Marvin Harrison Jr.  11  WR  16.8   7

In [16]:
!pip install pulp
import pulp

In [11]:
# Konfiguration
number_of_teams = 12
roster_requirements = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}
max_team_size = 15

# Initialisierung
teams = list(range(1, number_of_teams + 1))
rosters = {t: [] for t in teams}
roster_counts = {t: {pos: 0 for pos in roster_requirements} for t in teams}

# Draft-Reihenfolge (Snake Style)
draft_order = []
for r in range(max_team_size):
    draft_order += teams if r % 2 == 0 else reversed(teams)

# Schritt 1: Replacement-Level pro Position bestimmen
replacement_rank = {pos: number_of_teams * req for pos, req in roster_requirements.items()}

replacement_value = {}
for pos in roster_requirements:
    pos_players = df[df['POS'] == pos].sort_values(by="Value", ascending=False).reset_index(drop=True)
    idx = replacement_rank[pos] - 1  # Nullbasiert
    if idx < len(pos_players):
        replacement_value[pos] = pos_players.loc[idx, "Value"]
    else:
        replacement_value[pos] = pos_players["Value"].min()

# Schritt 2: VOR berechnen
df["VOR"] = df.apply(lambda row: row["Value"] - replacement_value[row["POS"]], axis=1)

# Datenvorbereitung
df["CombinedScore"] = 0.8 * df["Value"] + 0.2 * df["VOR"]
available_players = df.copy()
draft_history = []

# MIP-basierter Draft-Prozess
for pick_number, current_team in enumerate(draft_order, 1):
    # Aktuelle Team-Anforderungen
    current_roster = rosters[current_team]
    current_counts = roster_counts[current_team]

    # Positionsbedarf ermitteln
    needed_positions = [
        pos for pos, req in roster_requirements.items()
        if current_counts[pos] < req
    ]

    # Modell erstellen
    model = pulp.LpProblem(f"Draft_Pick_{pick_number}", pulp.LpMaximize)

    # Entscheidungsvariablen
    players = available_players['Player'].tolist()
    x = pulp.LpVariable.dicts("Draft", players, cat='Binary')

    # Zielfunktion (maximiere Combined Score)
    model += pulp.lpSum(
        x[p] * available_players.loc[available_players['Player'] == p, 'CombinedScore'].values[0]
        for p in players
    )

    # Nebenbedingungen
    ## Genau 1 Spieler pro Pick
    model += pulp.lpSum(x.values()) == 1

    ## Positionsbeschränkungen
    if needed_positions:
        model += pulp.lpSum(
            x[p] for p in available_players[
                available_players['POS'].isin(needed_positions)
            ]['Player']
        ) >= 1

    ## Gesamtteamgröße
    model += pulp.lpSum(x.values()) <= (max_team_size - len(current_roster))

    # Problem lösen
    model.solve(pulp.PULP_CBC_CMD(msg=0))

    # Ergebnis verarbeiten
    drafted_player = next(
        p for p in players
        if pulp.value(x[p]) == 1
    )

    player_data = available_players[available_players['Player'] == drafted_player].iloc[0]

    # Roster aktualisieren
    rosters[current_team].append({
        'Player': drafted_player,
        'POS': player_data['POS'],
        'Value': player_data['Value'],
        'VOR': player_data['VOR'],
        'CombinedScore': player_data['CombinedScore']
    })
    roster_counts[current_team][player_data['POS']] += 1

    # Spieler aus Available entfernen
    available_players = available_players[available_players['Player'] != drafted_player]

    # Draft-Historie speichern
    draft_history.append({
        'Pick': pick_number,
        'Team': current_team,
        'Player': drafted_player,
        'POS': player_data['POS'],
        'Value': player_data['Value'],
        'VOR': player_data['VOR']
    })

# Ergebnisse darstellen
draft_df = pd.DataFrame(draft_history)

print("\nDraft Verlauf:")
print(draft_df[['Pick', 'Team', 'Player', 'POS', 'Value']].to_string(index=False))

print("\nTeam Rosters:")
for team in teams:
    team_df = pd.DataFrame(rosters[team])
    team_value = team_df['Value'].sum()
    print(f"\nTeam {team} (Gesamtwert: {team_value:.1f}):")
    print(team_df[['Player', 'POS', 'Value']].to_string(index=False))


KeyboardInterrupt: 

In [17]:
print("Duplikate im Index:", df.index.has_duplicates)
print("NaN-Werte in AVG:", df['AVG'].isna().sum())
print("Duplikate in Player-Namen:", df['Player'].duplicated().sum())
print("Beispiel-Daten (erste 5 Zeilen):\n", df.head().to_string())


Duplikate im Index: False
NaN-Werte in AVG: 0
Duplikate in Player-Namen: 0
Beispiel-Daten (erste 5 Zeilen):
                 Player Bye POS  AVG        Value
0  Christian McCaffrey  9   RB  1.0  1000.000000
1          CeeDee Lamb   7  WR  2.6   968.506582
2          Tyreek Hill   6  WR  3.2   956.953957
3       Bijan Robinson  12  RB  5.0   923.116346
4          Breece Hall  12  RB  5.4   915.760877


In [18]:
import pulp
import pandas as pd
import numpy as np

# Konfiguration
number_of_teams = 12
roster_requirements = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}
max_team_size = 15
flex_positions = ['RB', 'WR', 'TE']
priority_positions = ['QB', 'RB', 'WR', 'TE']
late_round_threshold = 10  # Runde ab der K/DST gedraftet werden müssen

# Sicherheits-Check und Index-Reset
df = df.reset_index(drop=True).copy()
df = df.loc[~df.index.duplicated(keep='first')].copy()

def prepare_data(df):
    # Index zurücksetzen und Duplikate entfernen
    df = df.reset_index(drop=True)

    # Sicherstellen, dass keine doppelten Spieler vorhanden sind
    df = df.drop_duplicates(subset='Player', keep='first')

    # Dynamischer Replacement-Level
    df['ReplacementValue'] = (
        df.groupby('POS', as_index=False)['Value']
        .transform(lambda x: x.quantile(0.33 + 0.67*(1 - x.rank(pct=True))))
    )

    # VOR und Scores berechnen
    df['VOR'] = df['Value'] - df['ReplacementValue']
    df['RiskAdjScore'] = 0.7*df['Value'] + 0.3*df['VOR']
    df['ADP_Weight'] = np.clip(
        (df['AVG'] - df['AVG'].min()) / (df['AVG'].max() - df['AVG'].min()),
        0.2,
        1.0
    )

    return df.reset_index(drop=True)

# Initialisierung
df = prepare_data(df.copy())
teams = list(range(1, number_of_teams + 1))
rosters = {t: [] for t in teams}
roster_counts = {t: {pos: 0 for pos in roster_requirements} for t in teams}

# Snake-Draft-Reihenfolge
draft_order = []
for r in range(max_team_size):
    draft_order += teams if r % 2 == 0 else reversed(teams)

# MIP-basierter Draft-Prozess mit flexiblen Constraints
available_players = df.copy()
draft_history = []

for pick_number, current_team in enumerate(draft_order, 1):
    current_round = (pick_number - 1) // number_of_teams + 1
    current_roster = rosters[current_team]
    current_counts = roster_counts[current_team]

    # Positionsbedarf ermitteln (flexible Logik)
    needed_positions = []
    for pos, req in roster_requirements.items():
        remaining_picks = max_team_size - len(current_roster)
        remaining_required = req - current_counts[pos]

        # Für K/DST: Erst ab späten Runden erzwingen
        if pos in ['K', 'DST'] and current_round < late_round_threshold:
            remaining_required = max(0, remaining_required - 1)

        if remaining_required > 0 and remaining_picks >= remaining_required:
            needed_positions.append(pos)

    # Modell erstellen
    model = pulp.LpProblem(f"Draft_Pick_{pick_number}", pulp.LpMaximize)
    players = available_players['Player'].tolist()
    x = pulp.LpVariable.dicts("Draft", players, cat='Binary')

    # Zielfunktion mit dynamischer Gewichtung
    model += pulp.lpSum(
        x[p] * (
            available_players.loc[available_players['Player'] == p, 'RiskAdjScore'].values[0] *
            available_players.loc[available_players['Player'] == p, 'ADP_Weight'].values[0]
        )
        for p in players
    )

    # Constraints
    ## Genau 1 Spieler pro Pick
    model += pulp.lpSum(x.values()) == 1

    ## Positionsbeschränkungen (nur wenn notwendig)
    if needed_positions:
        pos_players = available_players[available_players['POS'].isin(needed_positions)]
        model += pulp.lpSum(x[p] for p in pos_players['Player']) >= 0.5  # Weiche Constraint

    ## Teamgröße
    model += pulp.lpSum(x.values()) <= (max_team_size - len(current_roster))

    ## ADP-Realismus
    if current_round <= 8: ###
        model += pulp.lpSum(
            x[p] * (available_players.loc[available_players['Player'] == p, 'AVG'].values[0] <= pick_number + 8)
            for p in players
        ) >= 0.5

    # Problem lösen
    model.solve(pulp.PULP_CBC_CMD(msg=0))

    # Ergebnis verarbeiten
    drafted_player = next(p for p in players if pulp.value(x[p]) == 1)
    player_data = available_players[available_players['Player'] == drafted_player].iloc[0]

    # Roster aktualisieren
    rosters[current_team].append({
        'Player': drafted_player,
        'POS': player_data['POS'],
        'Value': player_data['Value'],
        'VOR': player_data['VOR'],
        'ADP': player_data['AVG']
    })
    roster_counts[current_team][player_data['POS']] += 1

    # Verfügbare Spieler aktualisieren
    available_players = available_players[available_players['Player'] != drafted_player]

    # Draft-Historie
    draft_history.append({
        'Pick': pick_number,
        'Team': current_team,
        'Player': drafted_player,
        'POS': player_data['POS'],
        'Value': player_data['Value'],
        'ADP': player_data['AVG']
    })

# Nachträgliche Constraint-Überprüfung
for t in teams:
    for pos, req in roster_requirements.items():
        if roster_counts[t][pos] < req:
            # Erzwinge fehlende Positionen aus verfügbaren Spielern
            needed = req - roster_counts[t][pos]
            candidates = available_players[available_players['POS'] == pos]
            for _ in range(needed):
                if not candidates.empty:
                    best = candidates.nlargest(1, 'Value')
                    p = best['Player'].values[0]
                    rosters[t].append({
                        'Player': p,
                        'POS': pos,
                        'Value': best['Value'].values[0],
                        'VOR': best['VOR'].values[0],
                        'ADP': best['AVG'].values[0]
                    })
                    available_players = available_players[available_players['Player'] != p]

# Ergebnisse darstellen
draft_df = pd.DataFrame(draft_history)

print("\nDraft Verlauf:")
print(draft_df.sort_values('Pick')[['Pick', 'Team', 'Player', 'POS', 'ADP', 'Value']].to_string(index=False))

print("\nTeam Rosters:")
for team in teams:
    team_df = pd.DataFrame(rosters[team])
    team_value = team_df['Value'].sum()
    pos_counts = team_df['POS'].value_counts()

    print(f"\nTeam {team} (Gesamtwert: {team_value:.1f})")
    print("Positionen:")
    print(pos_counts.to_string())
    print("\nSpieler:")
    print(team_df[['Player', 'POS', 'ADP', 'Value']].sort_values(['POS', 'ADP']).to_string(index=False))


ValueError: cannot reindex on an axis with duplicate labels